In [2]:
import pandas as pd

### Data Collection

In [3]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module3_exercise_train.csv')
download_file(test_data_url, 'module3_exercise_test.csv')

Downloaded module3_exercise_train.csv from https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_train.csv
Downloaded module3_exercise_test.csv from https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_test.csv


In [4]:
df_train =  pd.read_csv("module3_exercise_train.csv", sep=",", index_col='id')

### Data Preprocessing

In [5]:
def data_prep(df):
    df['GarageCars'] = df['GarageCars'].replace({
    'zero': 0,
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4
    })
    garage_yr_mode = df.loc[df['GarageYrBlt'] != -1, 'GarageYrBlt'].mode()[0]
    df['GarageYrBlt'] = df['GarageYrBlt'].replace(-1, garage_yr_mode)
    return df

In [6]:
# Perform operations on the train dataset.
df_train = data_prep(df_train)
# df_train[df_train.isnull().any(axis=1)]
df_train['GarageCars'] = df_train['GarageCars'].fillna(2)  #Because the GarageArea value is around 408, properties generally have two garages.
df_train['YrSold'] = df_train['YrSold'].fillna(df_train['YrSold'].median())
df_train['TotRmsAbvGrd'] = df_train['TotRmsAbvGrd'].fillna(df_train['TotRmsAbvGrd'].median())
print("The number of missing values of train dataset:", df_train.isnull().sum().sum())

The number of missing values of train dataset: 0


/var/folders/4t/nwvmx6990tg417djct9q0qc00000gn/T/ipykernel_61890/3095565997.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['GarageCars'] = df['GarageCars'].replace({


### Model Building and Evaluation

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [8]:
y = df_train['SalePrice']
X = df_train[['BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
             'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
             'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
             'MoSold', 'YrSold']]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [11]:
def pred_eval(model, X_data, y_target):
    y_pred = model.predict(X_data)
    mae = mean_absolute_error(y_target, y_pred)
    r2 = r2_score(y_target, y_pred)
    print('R2 Score:\n', r2)
    print('Mean Absolute Error:\n', mae)
    return y_pred

In [12]:
y_pred_train = pred_eval(model, X_train, y_train)

R2 Score:
 0.6626841828283003
Mean Absolute Error:
 30940.43652218112


In [13]:
y_pred_test = pred_eval(model, X_test, y_test)

R2 Score:
 0.6460505733952246
Mean Absolute Error:
 31392.346575324882


In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [16]:
y_pred_train = pred_eval(model, X_train, y_train)

R2 Score:
 0.9580664872367434
Mean Absolute Error:
 10230.059389721626


In [17]:
y_pred_test = pred_eval(model, X_test, y_test)

R2 Score:
 0.6715186723434696
Mean Absolute Error:
 27270.90282051282


### Generating Submission File

In [18]:
X_test =  pd.read_csv("module3_exercise_test.csv", sep=",", index_col='id')

In [19]:
X_test = data_prep(X_test)
X_test[X_test.isnull().any(axis=1)]
X_test['EnclosedPorch'] = X_test['EnclosedPorch'].fillna(X_test['EnclosedPorch'].median())
X_test['Fireplaces'] = X_test['Fireplaces'].fillna(X_test['Fireplaces'].median())
print("The number of missing values of test dataset:", df_train.isnull().sum().sum())



The number of missing values of test dataset: 0


/var/folders/4t/nwvmx6990tg417djct9q0qc00000gn/T/ipykernel_61890/3095565997.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['GarageCars'] = df['GarageCars'].replace({


In [20]:
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X_test)


In [21]:
submission = pd.DataFrame({
    'id': X_test.index,
    'SalePrice': y_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

,id,SalePrice
0,892,279485.852306
1,1105,226818.631545
2,413,241866.156771
3,522,236504.164510
4,1036,167231.980058
